In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PyDAQmx as dx
import time



In [2]:
"This code is taken from PE2 experiment 2, from the file thermostat utensils"

def WriteVoltage(volt, myDAQchannel):
    #channel is a string being either AO0, AO1, DIO0 etc.
    analog_output = dx.Task()
    analog_output.CreateAOVoltageChan(myDAQchannel,"",-10.0,10.0,dx.DAQmx_Val_Volts,None)
    analog_output.StartTask()
    data = np.zeros((1,), dtype=np.float64)
    data[0] = volt
    pointsWritten = dx.int32()
    analog_output.WriteAnalogF64(1,0,10000.0,dx.DAQmx_Val_GroupByChannel,data,dx.byref(pointsWritten),None)
    analog_output.StopTask()
    
def ReadVoltage(myDAQchannel, myDAQ):    
    analog_input = dx.Task()
    analog_input.CreateAIVoltageChan(myDAQchannel,"",dx.DAQmx_Val_Cfg_Default,-10.0,10.0,dx.DAQmx_Val_Volts,None)
    analog_input.CfgSampClkTiming("",10000.0,dx.DAQmx_Val_Rising,dx.DAQmx_Val_FiniteSamps,2)

    # DAQmx Start Code
    analog_input.StartTask()
    read = dx.int32()
    data = np.zeros((2,), dtype=np.float64)
    analog_input.ReadAnalogF64(2,10.0,dx.DAQmx_Val_GroupByChannel,data,2,dx.byref(read),None)
    analog_input.StopTask()
    
def PIDFeedback(error_listing, proportional_constant, integral_constant, differential_constant):
    # Write desired output to heater
    if len(error_list >= 1):
        feedback_proportional = proportional_constant * error_listing[-1]
        feedback_integral = integral_constant * np.sum(previous_errors)
    if len(error_list) >= 2:
        #this is so that not everything will break during the first cycle.
        feedback_differential = differential_constant * (error_listing[-1] - error_listing[-2])
    else:
        feedback_differential = 0
    new_output = (-(feedback_proportional + feedback_integral + feedback_differential))
    
    
    return  new_output

    
    
def Wait(duration):
    start_time = time.time()
    delta_time = 0
    while delta_time < duration:
        delta_time = time.time() - start_time
    

In [10]:
class Piezo(object):
    
    def __init__(self, channel, volt_min, volt_max, volt_resolution, start_voltage):
        self.channel = channel
        self.volt_min = float(volt_min)
        self.volt_max = float(volt_max)
        self.volt_resolution = float(volt_resolution)
        
        if self.volt_max >= start_voltage >= self.volt_min:
            self.voltage = float(start_voltage)
            #WriteVoltage(start_voltage, channel)
        else:
            raise ValueError("the voltage is too below the minimum voltage or above the maximum voltage \n" + " Voltage: " + str(start_voltage) + "\n Min voltage: " + str(self.volt_min)  + "\n Max voltage: " + str(self.volt_max))
            self.voltage = self.volt_min

    def SetVoltage(self, new_voltage):
        """This is to add error checking, we don't want to break shit"""
        if self.volt_max > start_voltage > self.volt_min:
            self.voltage = new_voltage
            WriteVoltage(start_voltage, channel)
        else:
            raise ValueError("the voltage is too below the minimum voltage or above the maximum voltage \n" + " Voltage: " + str(self.voltage) + "\n Min voltage: " + str(self.volt_min)  + "\n Max voltage: " + str(self.volt_max))

        
    
        
    
class ScanningTunnelingMicroscope(object):
    
    def __init__(self, piezo_x, piezo_y, piezo_z, volt_range, error, target_voltage):
        self.piezo_x = piezo_x
        self.piezo_y = piezo_y
        self.piezo_z = piezo_z
        self.error = float(error)
        self.target_voltage = float(target_voltage)
        self.volt_range = float(volt_range) #this effectively decides how big of an area you're looking at.
        
        self.sample = np.zeros((int(self.volt_range / piezo_y.volt_resolution), int(self.volt_range / piezo_x.volt_resolution)))
    

    
          
    def ResetSample(self):
        self.SetSample(np.zeros((int(self.volt_range / piezo_y.volt_resolution), int(self.volt_range / piezo_x.volt_resolution))))
    
    def Measurement(self, read_channel, constant_prop, constant_int, constant_dif):
        
        time_start = time.time()
        time_elapsed = 0
        time_max = 5
        
        measured_error = 0
        measured_voltage = 0
        new_voltage = 0
        
        error_list = []
        
        target_volt = self.target_voltage()
        target_error = self.error

        
        z_piezo = self.piezo_z
        
        
        volt_min = z_piezo.volt_min
        
        measuring = True
        
        
        z_piezo.SetVoltage(volt_min)
        
        while measuring:
            measured_voltage = ReadVoltage(read_channel)
            measured_error = np.abs(target_volt - measured_voltage)
            
                
            if target_error >= measured_error or time_elapsed > time_max:
                measuring = False
            else:
                error_list.append(measured_error)
                new_voltage = PIDFeedback(error_list, constant_prop, constant_int, constant_dif)
                z_piezo.SetVoltage(new_voltage)
                time_elapsed = time.time() - time_start
        return measured_voltage
        
        
    def Measure_row(self, row, read_channel, constant_prop, constant_int, constant_dif):

        new_row = self.sample[row]
        target = self.target_voltage()

        y_piezo = self.piezo_y
        x_piezo =  self.piezo_x
        
        target_error = self.error
        
        volt_res = x_piezo.volt_resolution
        volt_min = x_piezo.volt_min
        
        y_piezo.SetVoltage(volt_min + volt_res * row)
        
        
        column_number = 0
        for column in new_row:
            x_piezo.SetVoltage(volt_min + volt_res * column_number)
            
            new_row[column_number] = self.Measure(read_channel, constant_prop, constant_int, constant_dif)
            
            column_number += 1
        self.sample[row] = new_row
    
    def Measure_sample(self, read_channel, constant_prop, constant_int, constant_dif):
        new_sample = self.sample
        
        for row in new_sample:
            new_sample[row] = Measure_row(row, read_channel, constant_prop, constant_int, constant_dif)
        
        self.sample = new_sample
            
    
        
        

In [11]:
#set the proper channels later
#all the input and output channels
channel_x = "myDAQ1/ao0"
channel_y = "myDAQ1/ao1"
channel_z = "myDAQ2/ao0"
channel_output = "myDAQ1/ai0"

#minimum voltage, resolution, start voltage and voltage range for the piezos
volt_min = 0
volt_max = 2
volt_res = 10e-3
volt_start = 0
voltage_range = 0.200

#Constants for the feedback like the target and error and linearity constants
error = 0.1
target = 5

const_prop = 1
const_int = 1
const_diff = 1

#Create the actual objects needed to build the STM and also build the STM itself
piezo_x = Piezo(channel_x, volt_min, volt_max, volt_res,volt_start)
piezo_y =  Piezo(channel_y, volt_min, volt_max, volt_res,volt_start)
piezo_z = Piezo(channel_z, volt_min, volt_max, volt_res,volt_start)

STM = ScanningTunnelingMicroscope(piezo_x, piezo_y, piezo_z, voltage_range, error, target)